In [1]:
"D:\pythonana\data-review-based-sa.csv"

'D:\\pythonana\\data-review-based-sa.csv'

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Step 1: Load and preprocess your dataset
# data = pd.read_csv('')  # Update with your dataset file path
# import dataset
pd.set_option('display.max_colwidth', 100)
data =pd.read_csv("D:\pythonana\data-review-based-sa.csv")
# Extract the text and label columns


In [4]:
from bs4 import BeautifulSoup
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Download the stop words corpus
# nltk.download('stopwords')

stops = set(stopwords.words('english')) #english stopwords

stemmer = SnowballStemmer('english') #SnowballStemmer

def review_to_words(raw_review):
    # 1. Delete HTML 
    try:
        review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
        # 2. Make a space
        letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
        # 3. lower letters
        words = letters_only.lower().split()
        # 5. Stopwords 
        meaningful_words = [w for w in words if not w in stops]
        # 6. Stemming
        stemming_words = [stemmer.stem(w) for w in meaningful_words]
        # 7. space join words
        return( ' '.join(stemming_words))
    except:
        return "i"

In [5]:
data['Review'] = data['Review'].apply(review_to_words)


C:\Users\LENOVO\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [6]:
# make proper modifications to the dataset
def giverate(i):
#     print(i)
#     if isdigit(i):
#         i=int(i)
    try:
        i=int(i)
    except ValueError:
        flag = False
    if i == 5:
        return 'positive'
    elif i==4:
        return 'semi-positive'
    elif i==3:
        return 'neutral'
    elif i==2:
        return 'semi-negative'
    else:
        return 'negative'

    
data['Sentiment'] = data['Rate'].apply(lambda x : giverate((x)))

In [7]:
data[:10
    ]

,product_name,product_price,Rate,Review,Summary,Sentiment
0,"Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling P...",3999,5,super,great cooler excellent air flow and for this price its so amazing and unbelievablejust love it,positive
1,"Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling P...",3999,5,awesom,best budget 2 fit cooler nice cooling,positive
2,"Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling P...",3999,3,fair,the quality is good but the power of air is decent,neutral
3,"Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling P...",3999,1,useless product,very bad product its a only a fan,negative
4,"Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling P...",3999,3,fair,ok ok product,neutral
5,"Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling P...",3999,5,awesom,the cooler is really fantastic and provides good air flow highly recommended,positive
6,"Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling P...",3999,5,high recommend,very good product,positive
7,"Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling P...",3999,3,nice,very nice,neutral
8,"Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling P...",3999,1,unsatisfactori,very bad cooler,negative
9,"Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling P...",3999,4,worth money,very good,semi-positive


In [8]:
texts = data['Summary'].tolist()

labels = data['Sentiment'].tolist()

# Step 2: Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Step 3: Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts)

# Step 4: Pad sequences to have equal length
max_len = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_len)

# Step 5: Split your dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, test_size=0.2, random_state=42)

# Step 6: Build the CNN model
embedding_dim = 100
filters = 64
kernel_size = 5

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Conv1D(filters, kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Step 7: Train the model
batch_size = 32
epochs = 10
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=epochs)

# Step 8: Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=-1)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Step 9: Decode the predicted labels
decoded_labels = label_encoder.inverse_transform(y_pred)


AttributeError: 'float' object has no attribute 'lower'

In [44]:
# Preprocess the incoming data
incoming_text = "value for money"
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Initialize the stemmer
stemmer = PorterStemmer()

# Example sentence
# sentence = "The quick brown foxes jumped over the lazy dogs"

# Tokenize the sentence
tokens = word_tokenize(incoming_text)

# Apply stemming to each token
incoming_text = " ".join([stemmer.stem(token) for token in tokens])

# Print the stemmed words
# print(incoming_text)
incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=max_len)

# Predict the sentiment label for the incoming data
predicted_label = np.argmax(model.predict(incoming_padded_sequence), axis=-1)

# Decode the predicted label
decoded_label = label_encoder.inverse_transform(predicted_label)[0]

print("Incoming Text:", incoming_text)
print("Predicted Label:", decoded_label)

ValueError: X has 65 features, but MultinomialNB is expecting 35460 features as input.

In [47]:
from numpy import *
import pickle
pickle.dump(model,open('model.pkl','wb'))
model=pickle.load(open("model.pkl",'rb'))

In [10]:
# Preprocess the incoming data
incoming_text = "value for money"
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Initialize the stemmer
stemmer = PorterStemmer()

# Example sentence
# sentence = "The quick brown foxes jumped over the lazy dogs"

# Tokenize the sentence
tokens = word_tokenize(incoming_text)

# Apply stemming to each token
incoming_text = " ".join([stemmer.stem(token) for token in tokens])

# Print the stemmed words
# print(incoming_text)
incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=max_len)

# Predict the sentiment label for the incoming data
predicted_label = np.argmax(model.predict(incoming_padded_sequence), axis=-1)

# Decode the predicted label
decoded_label = label_encoder.inverse_transform(predicted_label)[0]

print("Incoming Text:", incoming_text)
print("Predicted Label:", decoded_label)

NameError: name 'model' is not defined

In [62]:
import pickle

pickle.dump(model, open('cnn_model.pkl', 'wb'))


In [63]:
import pickle

mod = pickle.load(open('cnn_model.pkl', 'rb'))


In [64]:
import pickle
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the model from the pickle file
model = pickle.load(open('cnn_model.pkl', 'rb'))

# Preprocess the incoming data
incoming_text = "value for money"

# Initialize the stemmer
stemmer = PorterStemmer()

# Tokenize the sentence
tokens = word_tokenize(incoming_text)

# Apply stemming to each token
incoming_text = " ".join([stemmer.stem(token) for token in tokens])

# Preprocess the incoming text similar to the training data
incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=max_len)

# Predict the sentiment label for the incoming data
predicted_label = np.argmax(model.predict(incoming_padded_sequence), axis=-1)

# Decode the predicted label
decoded_label = label_encoder.inverse_transform(predicted_label)[0]

print("Incoming Text:", incoming_text)
print("Predicted Label:", decoded_label)


ValueError: X has 65 features, but MultinomialNB is expecting 35460 features as input.

In [59]:
dir(tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_keras_api_names',
 '_keras_api_names_v1',
 'analyzer',
 'char_level',
 'document_count',
 'filters',
 'fit_on_sequences',
 'fit_on_texts',
 'get_config',
 'index_docs',
 'index_word',
 'lower',
 'num_words',
 'oov_token',
 'sequences_to_matrix',
 'sequences_to_texts',
 'sequences_to_texts_generator',
 'split',
 'texts_to_matrix',
 'texts_to_sequences',
 'texts_to_sequences_generator',
 'to_json',
 'word_counts',
 'word_docs',
 'word_index']

In [55]:
type(tokenizer)

keras.preprocessing.text.Tokenizer

In [60]:
tokenizer.fit_on_sequences

<bound method Tokenizer.fit_on_sequences of <keras.preprocessing.text.Tokenizer object at 0x0000022CA108DEB0>>

In [65]:
import tensorflow as tf

# Save the model
tf.saved_model.save(model, 'saved_model')

# Load the model
loaded_model = tf.saved_model.load('saved_model')


ValueError: Expected an object of type `Trackable`, such as `tf.Module` or a subclass of the `Trackable` class, for export. Got MultinomialNB() with type <class 'sklearn.naive_bayes.MultinomialNB'>.